# Mixture of Gaussian with Constraint

In [37]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.cluster import KMeans
from scipy.stats import norm
import seaborn as sns; sns.set()

In [38]:
def init(n_cluster, std, X):
    Phi = np.full((n_cluster, ), 1 / n_cluster)
    kmeans = KMeans(n_clusters=n_cluster).fit(X[:, None])
    Mu = kmeans.cluster_centers_.reshape(-1,)
    return Phi, Mu 

In [39]:
def normal_pdf(x, mu, sigma):
    return 1.0 / (sigma * (2.0 * np.pi)**(1/2)) * np.exp(-1.0 * (x - mu)**2 / (2.0 * (sigma**2)))

In [40]:
def e_step(X, Phi, Mu, std):
    # w_ij
    W = np.zeros((len(X), len(Phi))) 
    for i, x in enumerate(X):
        probs = normal_pdf(x, Mu, std)
        W[i, :] = (Phi * probs) / np.sum(Phi * probs)
    W = np.nan_to_num(W)
    return W

In [41]:
def m_step(X, W):
    Phi = W.mean(axis=0)
    Mu = (W.T * X).T.sum(axis=0) / W.sum(axis=0)
    return Phi, Mu

In [42]:
def converged(old_Mu, Mu):
    return np.allclose(old_Mu, Mu, 0.0001)

In [43]:
def em(X, n_cluster, std):
    Phi, old_Mu = init(n_cluster, std, X)
    Mu = old_Mu + 1
    while not converged(old_Mu, Mu):
        W = e_step(X, Phi, Mu, std)
        old_Mu = Mu.copy()
        Phi, Mu = m_step(X, W)
        print(".", end='')
    return W, Phi, Mu

In [44]:
def log_likelihood(X, W, Mu, std):
    means = Mu[W.argmax(axis=1)]
    likes = normal_pdf(X, means, std)
    return sum(likes)

In [45]:
def GMM_constraint_fit():
    result = {}
    for d in range(6):
        visual_data = data[:, :, d]
        likelihood = np.zeros(len(data) * 150).reshape(len(data), 150)
        for i in range(len(data)):
            previous = 0
            for j in range(1, 151):
                W, Phi, Mu = em(visual_data[i].reshape(-1, ), j, 1)
                new = log_likelihood(visual_data[i].reshape(-1, ), W, Mu, 1)
                likelihood[i, j - 1] = new - previous
                previous = new
                print()
        result[str(d)] = np.argmax(likelihood, axis=1)
        print("one finished")
    df = pd.DataFrame(result)
    df.to_csv("GMM_fit_constraint.csv")
    return df

In [46]:
def extract_data():
    data = []
    for f in os.listdir("PCA_results/"):
        if not f.startswith(".") and f.startswith("sub"):
            data.append(pd.read_csv(os.path.join("PCA_results", f)).values)
    return np.stack(data)

In [47]:
data = extract_data()

In [ ]:
result_df = GMM_constraint_fit()